In [1]:
import os
import numpy as np
# import pandas as pd
import h5py
# from matplotlib import pyplot as plt
# %matplotlib inline
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import Normalizer
from sklearn.metrics import roc_auc_score
import my_callbacks
from keras.utils import np_utils
import my_metrics

from sklearn.utils import class_weight

Using TensorFlow backend.


In [2]:
# Where the .h5 data is stored
# b_h5 = '/eos/cms/store/user/fsiroky/hdf5_data/'
b_h5 = '/mnt/hdf5test/'
# b_h5   = '/home/test_local/'

pds  = {1: 'BTagCSV', 2: 'BTagMu', 3: 'Charmonium', 4:'DisplacedJet', 5: 'DoubleEG',
        6: 'DoubleMuon', 7: 'DoubleMuonLowMass',
       # 8: 'FSQJets', 9: 'HighMultiplicityEOF', #NOT ENOUGH DATA, NOTEBOOK FAILES
        10: 'HTMHT', 11: 'JetHT', 12: 'MET',
       # 13: 'MinimumBias', #NOT ENOUGH DATA
        14: 'MuonEG', 15: 'MuOnia',
       # 16: 'NoBPTX',
        17: 'SingleElectron', 18: 'SingleMuon', 19: 'SinglePhoton', 20: 'Tau', 21: 'ZeroBias'
}
nbr = 11

bg_files  = [b_h5+pds[nbr]+'_C_background.h5',b_h5+pds[nbr]+'_D_background.h5', b_h5+pds[nbr]+'_E_background.h5',
             b_h5+pds[nbr]+'_F_background.h5', b_h5+pds[nbr]+'_G_background.h5', b_h5+pds[nbr]+'_H_background.h5']

bg_jets   = [pds[nbr]+"_C_background", pds[nbr]+"_D_background", pds[nbr]+"_E_background",
             pds[nbr]+"_F_background", pds[nbr]+"_G_background", pds[nbr]+"_H_background"]

sig_files = [b_h5+pds[nbr]+'_C_signal.h5',b_h5+pds[nbr]+'_D_signal.h5', b_h5+pds[nbr]+'_E_signal.h5',
             b_h5+pds[nbr]+'_F_signal.h5', b_h5+pds[nbr]+'_G_signal.h5', b_h5+pds[nbr]+'_H_signal.h5']

sig_jets  = [pds[nbr]+"_C_signal", pds[nbr]+"_D_signal", pds[nbr]+"_E_signal",
             pds[nbr]+"_F_signal", pds[nbr]+"_G_signal", pds[nbr]+"_H_signal"]

good_jets = np.empty([0,2802])
bad_jets  = np.empty([0,2802])
def get_jets(bg_files, bg_jets, sig_files, sig_jets, ratio=1.0):
#     try:
#         bad_jets
#     except:
#         bad_jets  = np.empty([0,2802])
#     try:
#         good_jets
#     except:
#         good_jets = np.empty([0,2802])
    np.random.seed(24)
    for i in range(len(bg_files)):
        try:
            bg_jetfile  = h5py.File(bg_files[i],'r')
            bg_jet      = bg_jetfile[bg_jets[i]][:]
            sig_jetfile = h5py.File(sig_files[i],'r')
            sig_jet     = sig_jetfile[sig_jets[i]][:]
            global bad_jets
            global good_jets
            bad_jets    = np.concatenate((bad_jets, bg_jet), axis=0)
            try:
                ratio_abs   = ratio*len(bg_jet)/len(sig_jet)
                idx = np.arange(sig_jet.shape[0])
                mask = np.zeros_like(idx, dtype=bool)

                selected = np.random.choice(idx, len(bg_jet)*ratio, replace=False)
                mask[selected] = True

                good_jets = np.concatenate((good_jets, sig_jet[mask]), axis=0)
                print(np.sum(mask), len(bg_jet),  len(good_jets), ratio_abs) #FIX HERE
            except:
                print("Size of sig_jet of ", sig_jets[i], " is 0")
#                 ratio_abs = str("Size of sig_jet is " + len(sig_jet)) 

        except OSError:
            print("This Primary Dataset doesn't have ", bg_jets[i])
    return good_jets, bad_jets

      

def get_train_test_split(good_jets, bad_jets, ratio = None):
    x_train = np.concatenate((good_jets,bad_jets), axis=0)
    scaler = Normalizer()
    scaler.fit(x_train)
    x_train = scaler.transform(x_train)

    y_train = np.zeros(len(x_train))
    y_train[0:len(good_jets)]=1
    mask = shuffle((np.arange(len(x_train))), random_state=42)

    x_train = x_train[mask]
    y_train = y_train[mask]

    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=ratio, random_state=42)
    return x_train, x_test, y_train, y_test

In [3]:
%%time

ratios = [#0.5,1,3,
          1]
    

for pd in range (11,15):
    goodbad_ratio = []
    scores   = []
    roc_a    = []
    try:
        print(pds[pd])
    except KeyError:
        print("PD with number ", pd, " was commented out")
        continue
    #Ratios meant in the subsampling sense    
    for rat in range (0,len(ratios)):
    
        nbr = pd
        print("Primary dataset read and subsampled with ratio ", ratios[rat], " from" , b_h5, "is ", pds[nbr])
        bg_files  = [b_h5+pds[nbr]+'_C_background.h5',b_h5+pds[nbr]+'_D_background.h5', b_h5+pds[nbr]+'_E_background.h5',
                 b_h5+pds[nbr]+'_F_background.h5', b_h5+pds[nbr]+'_G_background.h5', b_h5+pds[nbr]+'_H_background.h5']

        bg_jets   = [pds[nbr]+"_C_background", pds[nbr]+"_D_background", pds[nbr]+"_E_background",
                 pds[nbr]+"_F_background", pds[nbr]+"_G_background", pds[nbr]+"_H_background"]

        sig_files = [b_h5+pds[nbr]+'_C_signal.h5',b_h5+pds[nbr]+'_D_signal.h5', b_h5+pds[nbr]+'_E_signal.h5',
                 b_h5+pds[nbr]+'_F_signal.h5', b_h5+pds[nbr]+'_G_signal.h5', b_h5+pds[nbr]+'_H_signal.h5']

        sig_jets  = [pds[nbr]+"_C_signal", pds[nbr]+"_D_signal", pds[nbr]+"_E_signal",
                 pds[nbr]+"_F_signal", pds[nbr]+"_G_signal", pds[nbr]+"_H_signal"]
    
        good_jets, bad_jets = get_jets(bg_files, bg_jets, sig_files, sig_jets, ratio=ratios[rat])
 
         

JetHT
Primary dataset read and subsampled with ratio  1  from /mnt/hdf5test/ is  JetHT
492 492 492 0.02854159415245388
121 121 613 0.004517959823762229
638 638 1251 0.04127312718333549
70 70 1321 0.003910396067258812
201 201 1522 0.005032548823234852
3104 3104 4626 0.07585718126053911
MET
Primary dataset read and subsampled with ratio  1  from /mnt/hdf5test/ is  MET
471 471 5097 0.028114367575956544
This Primary Dataset doesn't have  MET_D_background
901 901 5998 0.03675300836222721
63 63 6061 0.003714184648036788
189 189 6250 0.004608856808427624
This Primary Dataset doesn't have  MET_H_background
PD with number  13  was commented out
MuonEG
Primary dataset read and subsampled with ratio  1  from /mnt/hdf5test/ is  MuonEG
476 476 6726 0.027484265835209885
133 133 6859 0.004603350408417555
902 902 7761 0.03721122112211221
38 38 7799 0.002314109981121734
199 199 7998 0.004816769133949751
2817 2817 10815 0.06791715890734625
CPU times: user 5.54 s, sys: 9.26 s, total: 14.8 s
Wall time: 2m

In [4]:
       
x_train, x_test, y_train, y_test = get_train_test_split(good_jets, bad_jets, ratio = 0.15)
try:
    goodbad_ratio.append((len(good_jets)/(len(bad_jets)+len(good_jets))))
except:
    goodbad_ratio.append("Size of sig_jet is 0 ")
del(good_jets)
del(bad_jets)
# Second split for validation set
x_train, x_val, y_train, y_val   = train_test_split(x_train, y_train, test_size=0.15, random_state=1)

model = Sequential()
model.add(Dense(2000, input_dim=2802, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1000, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(500, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.1))
model.add(Dense(1, activation='sigmoid'))
# Optimize the optimizer if necessary
# adam = optimizers.Adam(#lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0 #defeault 'Adam'
# #                               lr=0.010, beta_1=0.8, beta_2=0.999, epsilon=1e-08, decay=0.01)
# )
model.compile(loss='binary_crossentropy',
          optimizer='Adam',
          metrics=['accuracy', my_metrics.f1, my_metrics.mcor])

class_weight = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
model.fit(x_train, y_train, 
          validation_data=(x_val,y_val),
          epochs=20,
          batch_size=256,
          class_weight = class_weight
          )

score = model.evaluate(x_test, y_test, batch_size=256)
y_pred = model.predict(x_test)
print()
roc_a.append(roc_auc_score(y_test, y_pred))
print("ROC AUC SCORE for ratio", ratios[rat], "for PD", pds[pd], "is", roc_a)

scores.append(score)
acc = [e[1] for e in scores]
print("ACCURACY for ratio", ratios[rat], "for PD", pds[pd], "is", acc)
print("GOOD/BAD ratio for ratio", ratios[rat], "for PD", pds[pd], "is", goodbad_ratio)
print("xxxxxxxxxx")
del(x_train)
del(x_test)
del(x_val)


Train on 15627 samples, validate on 2758 samples
Epoch 1/20
15627/15627 [==============================] - 22s - loss: 0.6915 - acc: 0.5172 - f1: 0.4540 - mcor: 0.0463 - val_loss: 0.6857 - val_acc: 0.5279 - val_f1: 0.6293 - val_mcor: 0.0664

NameError: name 'good_jets' is not defined